<a href="https://colab.research.google.com/github/CodeHunterOfficial/A_PythonLibraries/blob/main/%D0%91%D0%B8%D0%B1%D0%BB%D0%B8%D0%BE%D1%82%D0%B5%D0%BA%D0%B0_CatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Библиотека CatBoost для задач классификации

## Введение

CatBoost (Categorical Boosting) — это библиотека для градиентного бустинга, разработанная Яндексом. Она предназначена для работы с категориальными данными и обеспечивает высокую производительность при обработке больших наборов данных. В этой лекции мы подробно рассмотрим, как использовать CatBoost для решения задач классификации и регрессии, включая теоретические основы, функции потерь, метрики оценки, примеры использования и лучшие практики.

## 1. Основные понятия

### 1.1. Градиентный бустинг

Градиентный бустинг — это метод ансамблевого обучения, который строит модель, последовательно обучая слабые модели (обычно деревья решений) и корректируя ошибки предыдущих. Каждое новое дерево обучается на градиенте функции потерь от предсказаний предыдущих деревьев.

Формально, предсказание $F(x)$ для объекта $x$ можно выразить как:

$$
F(x) = F_{m-1}(x) + \nu \cdot h_m(x)
$$

где:
- $F_{m-1}(x)$ — предсказание предыдущей модели,
- $\nu$ — скорость обучения,
- $h_m(x)$ — m-ое дерево решений.

### 1.2. Потери для задач классификации

Для задач классификации используются различные функции потерь в зависимости от типа задачи:

- **Бинарная классификация:** логистическая регрессия. Функция потерь:

$$
L(y, F(x)) = - \left( y \log(\sigma(F(x))) + (1 - y) \log(1 - \sigma(F(x))) \right)
$$

где $\sigma$ — логистическая функция.

- **Мультиклассовая классификация:** кросс-энтропия. Функция потерь:

$$
L(y, F(x)) = - \sum_{c=1}^{C} y_c \log(\sigma(F_c(x)))
$$

где:
- $C$ — количество классов,
- $y_c$ — бинарный индикатор (1 или 0) для класса $c$,
- $\sigma(F_c(x))$ — вероятность того, что объект принадлежит классу $c$.

## 2. Установка и подготовка к работе с CatBoost

Для начала необходимо установить библиотеку. Это можно сделать с помощью pip:

```bash
pip install catboost
```

Импортируем необходимые библиотеки:

```python
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
```

## 3. Пример 1: Бинарная классификация

### 3.1. Подготовка данных

Рассмотрим набор данных "Titanic", в котором мы будем предсказывать выживание пассажиров.

```python
# Загрузка данных
data = pd.read_csv('titanic.csv')

# Выбор необходимых столбцов
data = data[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']]

# Обработка пропусков
data['Age'].fillna(data['Age'].median(), inplace=True)

# Кодирование категориальных признаков
data['Sex'] = data['Sex'].map({'male': 0, 'female': 1})

# Разделение данных на обучающую и тестовую выборки
X = data.drop('Survived', axis=1)
y = data['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
```

### 3.2. Обучение модели

Создаем и обучаем модель CatBoost:

```python
model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, verbose=100)
model.fit(X_train, y_train)
```

### 3.3. Оценка модели

Теперь оценим качество модели:

```python
y_pred = model.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Classification report
print(classification_report(y_test, y_pred))
```

### 3.4. Важность признаков

CatBoost позволяет легко визуализировать важность признаков:

```python
import matplotlib.pyplot as plt

feature_importances = model.get_feature_importance()
plt.barh(X.columns, feature_importances)
plt.xlabel('Importance')
plt.title('Feature Importance')
plt.show()
```

## 4. Пример 2: Мультиклассовая классификация

Рассмотрим набор данных "Iris", в котором мы будем предсказывать вид цветка.

### 4.1. Подготовка данных

```python
from sklearn.datasets import load_iris

# Загрузка данных
iris = load_iris()
X = iris.data
y = iris.target

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
```

### 4.2. Обучение модели

```python
model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, verbose=100)
model.fit(X_train, y_train)
```

### 4.3. Оценка модели

```python
y_pred = model.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Classification report
print(classification_report(y_test, y_pred))
```

### 4.4. Важность признаков

```python
feature_importances = model.get_feature_importance()
plt.barh(iris.feature_names, feature_importances)
plt.xlabel('Importance')
plt.title('Feature Importance')
plt.show()
```

## 5. Метрики для оценки качества классификации

### 5.1. Accuracy

Accuracy (доля правильных предсказаний) определяется как:

$$
\text{Accuracy} = \frac{TP + TN}{TP + TN + FP + FN}
$$

где:
- $TP$ — истинно положительные,
- $TN$ — истинно отрицательные,
- $FP$ — ложно положительные,
- $FN$ — ложно отрицательные.

### 5.2. Precision и Recall

- **Precision (точность)**:

$$
\text{Precision} = \frac{TP}{TP + FP}
$$

- **Recall (полнота)**:

$$
\text{Recall} = \frac{TP}{TP + FN}
$$

### 5.3. F1-score

F1-score — гармоническое среднее точности и полноты:

$$
F1 = 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}
$$

### 5.4. ROC и AUC

- **ROC (Receiver Operating Characteristic)** — кривая, показывающая соотношение между чувствительностью (Recall) и специфичностью (1 - FPR).
- **AUC (Area Under the Curve)** — площадь под кривой ROC. Чем выше AUC, тем лучше модель.

## 6. Обработка категориальных признаков

CatBoost может обрабатывать категориальные данные без необходимости их предварительного кодирования, что позволяет избежать переобучения и потери информации.

### Пример обработки категориальных признаков

```python
# Пример данных
data = pd.DataFrame({
    'Category': ['A', 'B', 'A', 'C', 'B', 'C', 'A', 'B'],
    'Value': [1, 2, 3, 4, 5, 6, 7, 8],
    'Target': [0, 1, 0, 1, 0, 1, 0, 1]
})

# Определяем категориальные признаки
cat_features = ['Category']

# Разделяем данные
X = data.drop('Target', axis=1)
y = data['Target']

# Создаем и обучаем модель
model = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=3, cat_features=cat_features, verbose=10)
model.fit(X, y)
```

## 7. Настройка гиперпараметров

### Пример настройки гиперпараметров с GridSearchCV

```python
from sklearn.model_selection import GridSearchCV

param_grid = {
    'iterations': [100, 200],
    'depth': [3, 6],
    'learning_rate': [0.1, 0.01],
}

grid_search = GridSearchCV(estimator=CatBoostClassifier(cat_features=cat_features, verbose=0),
                           param_grid=param_grid,
                           scoring='accuracy',
                           cv=3)

grid_search.fit(X, y)
print("Лучшие параметры:", grid_search.best_params_)
```

## 8. Кросс-валидация

### Пример кросс-валидации

```python
from catboost import cv, Pool

# Создаем Pool для кросс-валидации


pool = Pool(X, y, cat_features=cat_features)

# Параметры кросс-валидации
params = {
    'iterations': 100,
    'learning_rate': 0.1,
    'depth': 3,
    'eval_metric': 'Accuracy'
}

# Выполняем кросс-валидацию
cv_results = cv(params, pool, fold_count=5, shuffle=True, plot=True)
```


Давайте рассмотрим конкретные примеры использования библиотеки CatBoost для задач классификации, включая подготовку данных, обучение модели, оценку качества и визуализацию результатов.

## Пример 1: Бинарная классификация на основе данных о пассажирах "Титаника"

### 1. Подготовка данных

Сначала загрузим данные и проведем их предобработку. Мы будем использовать набор данных "Titanic", который доступен в Kaggle или других источниках.

#### Шаг 1: Загрузка необходимых библиотек

```python
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
```

#### Шаг 2: Загрузка и предобработка данных

```python
# Загрузка данных
data = pd.read_csv('titanic.csv')

# Выбор необходимых столбцов
data = data[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']]

# Обработка пропусков
data['Age'].fillna(data['Age'].median(), inplace=True)

# Кодирование категориальных признаков
data['Sex'] = data['Sex'].map({'male': 0, 'female': 1})

# Разделение данных на обучающую и тестовую выборки
X = data.drop('Survived', axis=1)
y = data['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.head())
```

### 2. Обучение модели

#### Шаг 3: Создание и обучение модели

```python
# Создаем и обучаем модель
model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, verbose=100)
model.fit(X_train, y_train)
```

### 3. Оценка качества модели

#### Шаг 4: Прогнозирование и оценка

```python
# Прогнозируем значения
y_pred = model.predict(X_test)

# Оценка качества
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Отчет по классификации
print(classification_report(y_test, y_pred))
```

### 4. Визуализация важности признаков

#### Шаг 5: Визуализация важности

```python
import matplotlib.pyplot as plt

# Получаем важность признаков
feature_importances = model.get_feature_importance()

# Строим график важности признаков
plt.barh(X.columns, feature_importances)
plt.xlabel('Importance')
plt.title('Feature Importance')
plt.show()
```

## Пример 2: Мультиклассовая классификация на основе данных о цветах "Iris"

### 1. Подготовка данных

Здесь мы используем набор данных Iris для предсказания вида цветка.

#### Шаг 1: Загрузка необходимых библиотек

```python
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report
```

#### Шаг 2: Загрузка и подготовка данных

```python
# Загрузка данных
iris = load_iris()
X = iris.data
y = iris.target

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
```

### 2. Обучение модели

#### Шаг 3: Создание и обучение модели

```python
# Создаем и обучаем модель
model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, verbose=100)
model.fit(X_train, y_train)
```

### 3. Оценка качества модели

#### Шаг 4: Прогнозирование и оценка

```python
# Прогнозируем значения
y_pred = model.predict(X_test)

# Оценка качества
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Отчет по классификации
print(classification_report(y_test, y_pred))
```

### 4. Визуализация важности признаков

#### Шаг 5: Визуализация важности

```python
import matplotlib.pyplot as plt

# Получаем важность признаков
feature_importances = model.get_feature_importance()

# Строим график важности признаков
plt.barh(iris.feature_names, feature_importances)
plt.xlabel('Importance')
plt.title('Feature Importance')
plt.show()
```

## Пример 3: Классификация клиентов в банке

В этом примере мы будем предсказывать, уйдет ли клиент из банка, используя вымышленные данные о клиентах.

### 1. Подготовка данных

#### Шаг 1: Загрузка необходимых библиотек

```python
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
```

#### Шаг 2: Создание вымышленного набора данных

```python
# Создаем вымышленный набор данных
data = pd.DataFrame({
    'CustomerID': range(1, 101),
    'Age': [25, 30, 35, 40, 45] * 20,
    'Gender': ['Male', 'Female'] * 50,
    'Region': ['North', 'South', 'East', 'West'] * 25,
    'Churn': [0, 1, 0, 1, 0] * 20
})

# Кодируем категориальные признаки
data['Gender'] = data['Gender'].map({'Male': 0, 'Female': 1})
data['Region'] = data['Region'].astype('category')

# Разделяем данные
X = data.drop(['CustomerID', 'Churn'], axis=1)
y = data['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
```

### 2. Обучение модели

#### Шаг 3: Создание и обучение модели

```python
# Создаем и обучаем модель
cat_features = ['Region']
model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=4, cat_features=cat_features, verbose=100)
model.fit(X_train, y_train)
```

### 3. Оценка качества модели

#### Шаг 4: Прогнозирование и оценка

```python
# Прогнозируем значения
y_pred = model.predict(X_test)

# Оценка качества
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Отчет по классификации
print(classification_report(y_test, y_pred))
```

### 4. Визуализация важности признаков

#### Шаг 5: Визуализация важности

```python
import matplotlib.pyplot as plt

# Получаем важность признаков
feature_importances = model.get_feature_importance()

# Строим график важности признаков
plt.barh(X.columns, feature_importances)
plt.xlabel('Importance')
plt.title('Feature Importance')
plt.show()
```


Таким образом, мы рассмотрели основные аспекты работы с библиотекой CatBoost для задач классификации. Мы изучили:

- Теоретические основы градиентного бустинга и функции потерь для классификации.
- Как установить и подготовить библиотеку CatBoost.
- Примеры бинарной и мультиклассовой классификации.
- Важность признаков и метрики для оценки качества модели.
- Обработку категориальных признаков, настройку гиперпараметров и кросс-валидацию.





# Библиотека CatBoost для задач регрессии


## 1. Основные понятия

###  Потери для задач регрессии

Для задач регрессии используются различные функции потерь в зависимости от задачи:

- **Среднеквадратичная ошибка (MSE)**. Функция потерь:

$$
L(y, F(x)) = \frac{1}{n} \sum_{i=1}^{n} (y_i - F(x_i))^2
$$

где $y$ — истинные значения, $F(x)$ — предсказанные значения.

- **Средняя абсолютная ошибка (MAE)**. Функция потерь:

$$
L(y, F(x)) = \frac{1}{n} \sum_{i=1}^{n} |y_i - F(x_i)|
$$

## 2. Установка и подготовка к работе с CatBoost

Для начала необходимо установить библиотеку. Это можно сделать с помощью pip:

```bash
pip install catboost
```

Импортируем необходимые библиотеки:

```python
import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
```

## 3. Пример 1: Регрессия

### 3.1. Подготовка данных

Рассмотрим набор данных "California housing", в котором мы будем предсказывать среднюю цену жилья.

```python
from sklearn.datasets import fetch_california_housing

# Загрузка данных
data = fetch_california_housing()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
```

### 3.2. Обучение модели

Создаем и обучаем модель CatBoost:

```python
model = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=6, verbose=100)
model.fit(X_train, y_train)
```

### 3.3. Оценка модели

Теперь оценим качество модели:

```python
y_pred = model.predict(X_test)

# Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')

# Mean Absolute Error
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae:.2f}')
```

### 3.4. Важность признаков

CatBoost позволяет легко визуализировать важность признаков:

```python
import matplotlib.pyplot as plt

feature_importances = model.get_feature_importance()
plt.barh(X.columns, feature_importances)
plt.xlabel('Importance')
plt.title('Feature Importance')
plt.show()
```

## 4. Метрики для оценки качества регрессии

### 4.1. Среднеквадратичная ошибка (MSE)

Среднеквадратичная ошибка (MSE) определяется как:

$$
\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
$$

где:
- $y_i$ — истинные значения,
- $\hat{y}_i$ — предсказанные значения.

### 4.2. Средняя абсолютная ошибка (MAE)

Средняя абсолютная ошибка (MAE) определяется как:

$$
\text{MAE} = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y}_i|
$$

## 5. Обработка категориальных признаков

CatBoost может обрабатывать категориальные данные без необходимости их предварительного кодирования, что позволяет избежать переобучения и потери информации.

### Пример обработки категориальных признаков

```python
# Пример данных
data = pd.DataFrame({
    'Category': ['A', 'B', 'A', 'C', 'B', 'C', 'A', 'B'],
    'Value': [1, 2, 3, 4, 5, 6, 7, 8],
    'Target': [0.5, 1.0, 0.6, 1.5, 0.9, 1.2, 0.7, 1.0]
})

# Определяем категориальные признаки
cat_features = ['Category']

# Разделяем данные
X = data.drop('Target', axis=1)
y = data['Target']

# Создаем и обучаем модель
model = CatBoostRegressor(iterations=100, learning_rate=0.1, depth=3, cat_features=cat_features, verbose=10)
model.fit(X, y)
```

## 6. Настройка гиперпараметров

### Пример настройки гиперпараметров с GridSearchCV

```python
from sklearn.model_selection import GridSearchCV

param_grid = {
    'iterations': [100, 200],
    'depth': [3, 6],
    'learning_rate': [0.1, 0.01],
}

grid_search = GridSearchCV(estimator=CatBoostRegressor(cat_features=cat_features, verbose=0),
                           param_grid=param_grid,
                           scoring='neg_mean_squared_error',
                           cv=3)

grid_search.fit(X, y)
print("Лучшие параметры:", grid_search.best_params_)
```

## 7. Кросс-валидация

### Пример кросс-валидации

```python
from catboost import cv, Pool

# Создаем Pool для кросс-валидации
pool = Pool(X, y, cat_features=cat_features)

# Параметры кросс-валидации
params = {
    'iterations': 100,
    'learning_rate': 0.1,
    'depth': 3,
    'eval_metric': 'RMSE'
}

# Выполняем кросс-валидацию
cv_results = cv(params, pool, fold_count=5, shuffle=True, plot=True)
```

 Давайте разберем несколько конкретных примеров использования CatBoost для решения задач регрессии. Мы будем использовать набор данных "California housing", где предсказывается средняя цена жилья, а также рассмотрим набор данных с вымышленными данными для демонстрации обработки категориальных признаков.

## Пример 1: Регрессия на наборе данных California Housing

### 1. Подготовка данных

Загрузим данные, разделим их на обучающую и тестовую выборки, и проведем предварительную обработку.

```python
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

# Загрузка данных
data = fetch_california_housing()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Проверка размеров выборок
print(f'Размер обучающей выборки: {X_train.shape[0]}')
print(f'Размер тестовой выборки: {X_test.shape[0]}')
```

### 2. Обучение модели

Создадим и обучим модель CatBoostRegressor.

```python
from catboost import CatBoostRegressor

# Создаем и обучаем модель
model = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=6, verbose=100)
model.fit(X_train, y_train)
```

### 3. Оценка модели

Оценим качество модели с помощью среднеквадратичной ошибки (MSE) и средней абсолютной ошибки (MAE).

```python
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Предсказания на тестовой выборке
y_pred = model.predict(X_test)

# Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')

# Mean Absolute Error
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae:.2f}')
```

### 4. Важность признаков

Визуализируем важность признаков для нашей модели.

```python
import matplotlib.pyplot as plt

# Получаем важность признаков
feature_importances = model.get_feature_importance()
plt.barh(X.columns, feature_importances)
plt.xlabel('Importance')
plt.title('Feature Importance')
plt.show()
```

## Пример 2: Регрессия с категориальными признаками

Для этого примера создадим набор данных с вымышленными данными, включая категориальные признаки.

### 1. Подготовка данных

Создадим DataFrame с категориальными и числовыми признаками.

```python
import numpy as np

# Создаем набор данных
data = pd.DataFrame({
    'Category': ['A', 'B', 'A', 'C', 'B', 'C', 'A', 'B', 'A', 'C'],
    'Value': np.random.rand(10) * 10,
    'Target': np.random.rand(10) * 100
})

# Проверяем данные
print(data)
```

### 2. Обработка категориальных признаков

Определяем категориальные признаки и разделяем данные на обучающую и тестовую выборки.

```python
# Определяем категориальные признаки
cat_features = ['Category']

# Разделяем данные
X = data.drop('Target', axis=1)
y = data['Target']

# Разделяем на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
```

### 3. Обучение модели

Обучаем модель, указывая, какие признаки являются категориальными.

```python
# Создаем и обучаем модель
model = CatBoostRegressor(iterations=100, learning_rate=0.1, depth=3, cat_features=cat_features, verbose=10)
model.fit(X_train, y_train)
```

### 4. Оценка модели

Оцениваем качество модели, как и в предыдущем примере.

```python
# Предсказания на тестовой выборке
y_pred = model.predict(X_test)

# Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')

# Mean Absolute Error
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae:.2f}')
```

### 5. Важность признаков

Визуализируем важность признаков для модели с категориальными данными.

```python
# Получаем важность признаков
feature_importances = model.get_feature_importance()
plt.barh(X.columns, feature_importances)
plt.xlabel('Importance')
plt.title('Feature Importance')
plt.show()
```




Таким образом, мы рассмотрели основные аспекты работы с библиотекой CatBoost для задач регрессии. Мы изучили:

- Теоретические основы градиентного бустинга и функции потерь для регрессии.
- Как установить и подготовить библиотеку CatBoost.
- Примеры регрессии на наборе данных California housing.
- Важность признаков и метрики для оценки качества модели.
- Обработку категориальных признаков, настройку гиперпараметров и кросс-валидацию.
